# Salary Prediction from LinkedIn Job Postings - Train Linear Model

In [ ]:
import pandas as pd, numpy as np
import salary
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

/var/folders/vm/77wrgjgj5wzbyghx353b7gym0000gn/T/ipykernel_85135/2006324968.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd, numpy as np


## Train & Evaluate Model

### Target normal salaries

In [18]:
model = salary.train_evaluate_model(make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    LinearRegression()
))

Mean CV train R2: 0.2905
Mean CV test R2: 0.2852
Mean CV train RMSE: 51285.1811
Mean CV test RMSE: 51481.2437
Mean CV train MAE: 33646.5948
Mean CV test MAE: 33765.6473
On average, our predicted salaries are $33765.65 off the true salaries


### Target log salaries

In [19]:
model_log = salary.train_evaluate_model(make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    TransformedTargetRegressor(
        regressor=LinearRegression(),
        func=np.log10,
        inverse_func=lambda x: 10**x
    )
))

Mean CV train R2: 0.2674
Mean CV test R2: 0.2617
Mean CV train RMSE: 52114.1258
Mean CV test RMSE: 52322.7680
Mean CV train MAE: 32355.7292
Mean CV test MAE: 32475.4911
On average, our predicted salaries are $32475.49 off the true salaries


In [22]:
best_model = model
best_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('target_encoder',
                                                  TargetEncoder(),
                                                  ['norm_title',
                                                   'location_state',
                                                   'company_industries']),
                                                 ('experience_level',
                                                  OrdinalEncoder(categories=[['Unknown',
                                                                              'Internship',
                                                                              'Entry '
                                                                              'level',
                                                                              'Associate',
                                                                              'Mid-Senior '
                                                                              'level',
                                                                              'Director',
                                                                              'Executive']]),
                                                  ['formatted_experience_level']),
                                                 ('work_type',
                                                  OrdinalEncoder(categories=[['Other',
                                                                              'Volunteer',
                                                                              'Internship',
                                                                              'Temporary',
                                                                              'Part-time',
                                                                              'Contract',
                                                                              'Full-time']]),
                                                  ['formatted_work_type']),
                                                 ('remote_allowed',
                                                  'passthrough',
                                                  ['remote_allowed']),
                                                 ('company_employee_count',
                                                  SimpleImputer(strategy='median'),
                                                  ['company_employee_count'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

## Feature Importance

In [21]:
feature_names = best_model[:-1].get_feature_names_out(salary.df_X.columns)

def get_important_features(coef, n=10):
    idxs = np.flip(np.argsort(np.abs(coef))[-n:])
    return pd.DataFrame([feature_names[idxs], coef[idxs]], index=["Feature", "Coefficient"]).transpose()

get_important_features(best_model[-1].coef_)

,Feature,Coefficient
0,target_encoder__norm_title,16881.847526
1,target_encoder__company_industries,13444.209279
2,experience_level__formatted_experience_level,9477.919424
3,target_encoder__location_state,8459.760909
4,work_type__formatted_work_type,4062.875979
5,company_employee_count__company_employee_count,3911.625007
6,remote_allowed__remote_allowed,1138.654533
